In [12]:
import main

In [17]:
X_train = main.X_train_stack
X_test = main.X_test_stack

In [18]:
# import libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

In [19]:
# for modeling 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.callbacks import EarlyStopping 

# Data Preparation

In [20]:
X_data_train = X_train.iloc[:, 0:9]
y_data_train = X_train.iloc[:,9:10]

In [21]:
X_data_test = X_test.iloc[:, 0:9]
y_data_test = X_test.iloc[:, 9:10]

In [22]:
# resize 
X_data_train = np.resize(X_data_train, (X_data_train.shape[0], X_data_train.shape[1]))
X_data_test = np.resize(X_data_test, (X_data_test.shape[0], X_data_test.shape[1]))

In [23]:
# normalizer 
scaler = Normalizer().fit(X_data_train)
X_data_train = scaler.transform(X_data_train)
scaler = Normalizer().fit(X_data_test)
X_data_test = scaler.transform(X_data_test)

In [24]:
# standard scaler 
s = StandardScaler()
s.fit(X_data_train)
X_data_train = s.transform(X_data_train)
X_data_test = s.transform(X_data_test)

In [25]:
# features for cnn to 10 features:
X_train = np.resize(X_train, (X_train.shape[0], 9))
X_test = np.resize(X_test, (X_test.shape[0], 9))

In [26]:
X_train.shape[1]

9

# RNN Model

In [7]:
# define the Recurrent Neural Network model 
from torch import nn 
import torch.nn.functional as F 
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# declare hyper-parameters
sequence_length = 28 
input_size = 28 
hidden_size = 128 
num_layers = 2
num_classes = 2
batch_size = 100
num_epochs = 2
learning_rate = 0.01 

In [8]:
class RNN(nn.Module):
    pass 

model = RNN().to(device)

In [9]:
print(model)

RNN()


In [10]:
# step 2: add constructor in RNN class 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
        pass
    
    def forward(self, x):
        # set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # passing in the input and hidden state into the model and obtain outputs 
        out, hidden = self.lstm(x, (h0, c0))    # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # reshaping the outputs such that it can be fit into the fully connected layer 
        out = self.fc(out[:, -1, :])
        
        return out 
        
    pass

pass 

In [13]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [14]:
loss_func = nn.CrossEntropyLoss()

In [15]:
from torch import optim 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [3]:
# compile the model 
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

NameError: name 'model' is not defined

In [4]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

NameError: name 'EarlyStopping' is not defined

In [ ]:
history = model.fit(X_data_train, y_data_train, callbacks=[es], epochs=10, batch_size=10, validation_split=0.2, shuffle=True, verbose=1)

Epoch 1/10
 57321/232614 [======>.......................] - ETA: 21:08 - loss: 7.4468e-04 - accuracy: 0.9997

In [ ]:
from matplotlib import pyplot 

# plot diagnostic learning curves
def summarize_diagnostic(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    
    # save plot to file 
    pyplot.savefig('cnn_performance_plot.png')
    pyplot.close()

In [ ]:
summarize_diagnostic(history)